Overpass Turbo Request: https://overpass-turbo.eu/s/1g8P

In [2]:
import requests
import io
import pandas as pd
import geojson


overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:csv(::id,::type,name,::lat,::lon,wikidata,wikipedia,website,email,opening_hours;true; ";")];
area["ISO3166-1"="CH"][admin_level=2];
(
  node["bath:open_air"="yes"](area);
  way["bath:open_air"="yes"](area);
  rel["bath:open_air"="yes"](area);
);
out center;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.content

print (data)

df = pd.read_csv(io.StringIO(data.decode('utf-8')), delimiter=";")

# https://stackoverflow.com/questions/47308984/how-to-use-python-with-pandas-exporting-data-to-geojson-for-google-map
# https://gist.github.com/mapcloud/036688121a87f960e35cc03077001e93
# https://shouland.com/pandas-to-geojson-multiples-points-features-with-python

# Replace nan to None
dataFrame = df.where((pd.notnull(df)), None)
dataFrame.rename(columns={'@lat': 'lat'}, inplace=True)
dataFrame.rename(columns={'@lon': 'lon'}, inplace=True)
dataFrame.rename(columns={'@id': 'osm_id'}, inplace=True)

def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["lon"], X["lat"])),
                            properties=dict(osm_id=X["osm_id"],name=X["name"], wikidata=X["wikidata"], wikipedia=X["wikipedia"], website=X["website"], email=X["email"], opening_hours=X["opening_hours"])))
    df.apply(insert_features, axis=1)
    with open('swimming.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, indent=4, sort_keys=True, ensure_ascii=False)

data2geojson(dataFrame)

b'@id;@type;name;@lat;@lon;wikidata;wikipedia;website;email;opening_hours\n52200228;node;Arenenberger Badi;47.6740707;9.0656653;;;;;\n277629858;node;Strandbad M\xc3\xbchlehorn;47.1180505;9.1744550;;;;;\n456036314;node;Freibad Teufen;47.3834052;9.3938663;;;http://www.teufen.ch/xml_1/internet/de/application/d361/d120/f121.cfm;;\n631879657;node;Badeinseln;46.8944453;8.6071003;;;;;\n1072932563;node;Seebad Baldegg;47.1862200;8.2770160;;;http://seebad-baldegg.ch/;;\n1080951326;node;Alte Badi Immensee;47.0996980;8.4690960;;;;;\n1728558194;node;Freibad Lerchenfeld;47.4158291;9.3417313;;;;;\n2218035420;node;Therme Vals;46.6220041;9.1812710;Q675081;de:Therme Vals;http://www.therme-vals.ch;;\n2327477051;node;Strandbad Nottwil;47.1411850;8.1361054;;;;;\n2573478863;node;Strandbad Lachen;47.1970270;8.8523842;;;;;\n2843590786;node;Seebad Gelfingen;47.2128790;8.2633525;;;;;Mo-Su 06:00-22:00\n3008686174;node;Schwimmbad Waldstatt;47.3540911;9.2809147;;;https://www.waldstatt.ch/unser-dorf/freizeit-sporta

ValueError: Out of range float values are not JSON compliant: nan